## **Step 1: Setup the Environment**

In [1]:
!pip install langchain
!pip install transformers
!pip install faiss-cpu  # For similarity search
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mult

Some important sub fucntions might not be downloaded or install properly therefore perform the given command

In [11]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00


This library is mainly used to create embeddings

In [15]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.7 MB/s eta 0:00:00


## **Step 2: Import Libraries**

Import the necessary libraries into your notebook:

In [12]:
import os
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from datasets import load_dataset
from transformers import pipeline


## **Step 3: Load the Dataset**

Load the dataset from Hugging Face using the datasets library:

In [17]:
from langchain.schema import Document  # Import Document class

# Load the dataset from Hugging Face
dataset = load_dataset("parsi-ai-nlpclass/Psychology_RAG")

# Convert to a list of documents using the Document class
documents = [
    Document(page_content=item["question"], metadata={"answer": item["answer"]})
    for item in dataset["train"]
]


## **Step 4: Create the Vector Store**

Create a vector store using FAISS with Hugging Face embeddings:

In [18]:
# Initialize the Hugging Face embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create the FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings)


In [19]:
# Create the FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings)


## **Step 5: Set Up the Language Model**

Instead of using OpenAI, use a Hugging Face model for generating responses. Here, we can use a text generation model like distilgpt2:

In [23]:
# Initialize the Hugging Face text generation pipeline
generator = pipeline("text-generation", model="distilgpt2",max_new_tokens=50)

# Create a custom HuggingFacePipeline for LangChain
llm = HuggingFacePipeline(pipeline=generator)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## **Step 6: Set Up the RetrievalQA Chain**

Set up the RetrievalQA chain using the vector store and the language model:

In [24]:
# Create the RetrievalQA chain
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

## **Step 7: Query the Model**

Now you can use the model to answer questions based on the dataset:

In [26]:
# Ask a question
query = "A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show."
response = retrieval_qa.run(query)

print("Response:", response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show. She becomes extremely distressed when her daily routine is disrupted.

A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show. She becomes extremely distressed when her daily routine is disrupted.

A 7-year-old girl has significant delays in language development and prefers to play alone, often spinning objects repeatedly.

A 7-year-old girl has significant delays in language development and prefers to play alone, often spinning objects repeatedly.

Question: A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show.
Helpful Answer: A 7-year-old girl currently needs medication to help trea